In [1]:

# Run these in the RGui first
# install.packages("devtools")
# library(devtools)
# install.packages("Rcpp")
# devtools::install_github("mkearney/rtweet")
library(rtweet)

In [3]:

# Run this in the RGui first
# install.packages("tidyverse")
library(tidyverse)

-- Attaching packages ------------------------------------------------------------------------------- tidyverse 1.3.0 --

v ggplot2 3.3.3     v purrr   0.3.4
v tibble  3.0.5     v dplyr   1.0.3
v tidyr   1.1.2     v stringr 1.4.0
v readr   1.4.0     v forcats 0.5.0

-- Conflicts ---------------------------------------------------------------------------------- tidyverse_conflicts() --
x dplyr::filter()  masks stats::filter()
x purrr::flatten() masks rtweet::flatten()
x dplyr::lag()     masks stats::lag()



In [5]:

# Run this in the RGui first
# install.packages("vegan")
library(vegan)

Loading required package: permute

Loading required package: lattice

This is vegan 2.5-7



In [7]:

# Run this in the RGui first
# install.packages("Rtsne")
library(Rtsne)

In [9]:

source("../R/FastPCA.R")

In [10]:

# Run this in the RGui first
# install.packages("fastcluster")
library(fastcluster)


Attaching package: 'fastcluster'


The following object is masked from 'package:stats':

    hclust




In [12]:

# Run this in the RGui first
# install.packages("dendextend")
library(dendextend)

Registered S3 method overwritten by 'dendextend':
  method     from 
  rev.hclust vegan


---------------------
Welcome to dendextend version 1.14.0
Type citation('dendextend') for how to cite the package.

Type browseVignettes(package = 'dendextend') for the package vignette.
The github page is: https://github.com/talgalili/dendextend/

Suggestions and bug-reports can be submitted at: https://github.com/talgalili/dendextend/issues
Or contact: <tal.galili@gmail.com>

	To suppress this message use:  suppressPackageStartupMessages(library(dendextend))
---------------------



Attaching package: 'dendextend'


The following object is masked from 'package:permute':

    shuffle


The following object is masked from 'package:stats':

    cutree




In [13]:

# ggplot2 colors
gg_color_hue <- function(n) {
  hues = seq(15, 375, length = n + 1)
  hcl(h = hues, l = 65, c = 100)[1:n]
}

# Turn off annoying warning
options(dplyr.summarise.inform=F)

In [28]:

require(jsonlite)
filepath<-"../data/secrets/client_secrets.json"
json_data <- fromJSON(paste(readLines(filepath), collapse=""))

In [29]:

# Twitter API authorization
api_key = json_data$twitter$api_key
api_secret = json_data$twitter$api_secret
access_token = json_data$twitter$access_token
access_secret = json_data$twitter$access_secret
app_name = json_data$twitter$app_name

create_token(
  app = app_name,
  consumer_key = api_key,
  consumer_secret = api_secret,
  access_token = access_token,
  access_secret = access_secret
)

<Token>
<oauth_endpoint>
 request:   https://api.twitter.com/oauth/request_token
 authorize: https://api.twitter.com/oauth/authenticate
 access:    https://api.twitter.com/oauth/access_token
<oauth_app> texasflooding
  key:    528AVTndAszHfs4AElFBO23do
  secret: <hidden>
<credentials> oauth_token, oauth_token_secret
---

In [31]:

# Start collecting data
fuser.favs = list()

twuser = "dbabbitt" # User around which to collect neighborhood
fuser.favs[[twuser]] = get_favorites(twuser, n=3000)
fuser.data = users_data(fuser.favs[[twuser]])

i=1
fusers = unique(fuser.favs[[twuser]]$screen_name)

ERROR: Error: Not a valid access token.


In [ ]:

while(i <= length(fusers)) {
  
  fuser = fusers[i]
  if(fuser %in% names(fuser.favs)) { i = i+1; next }
  print(fuser)
  
  # get data from twitter API
  tmp.favs = tibble()
  tc = try({
    tmp.favs = get_favorites(fuser, n=3000)
  })

  # inelegant way to retry when API limits exceeded
  if((class(tc)[1] != "try-error") & (class(tc)[1] != "try-warning")) {
    # add to dataset
    fuser.favs[[fuser]] = tmp.favs
    i = i + 1
    
    # reformat data
    fu.all = tibble()
    for(fu in names(fuser.favs)) {
      if(nrow(fuser.favs[[fu]]) == 0) next
      fu.grp = fuser.favs[[fu]] %>%
        group_by(screen_name) %>%
        summarise(n = n())
      fu.grp$user = fu
      fu.all = bind_rows(fu.all,fu.grp)
    }
    
    fu.mat = fu.all %>% pivot_wider(names_from = user, values_from = n, values_fn = list(n = sum))
    fu.mat[is.na(fu.mat)] = 0
    fu.df = as.data.frame(fu.mat)
    rownames(fu.df) = fu.df[,1]
    fu.df$screen_name = NULL
    fu.m = as.matrix(fu.df)
    
    # clustering based on who likes users
    # fu.m = fu.m[rowSums(fu.m>0)>30,colSums(fu.m>0)>5]
    # fu.m = fu.m[rowSums(fu.m>0)>30,colSums(fu.m>0)>5]
    # fu.m = fu.m[rowSums(fu.m>0)>30,colSums(fu.m>0)>5]
    # fu.m = fu.m[rowSums(fu.m>0)>30,colSums(fu.m>0)>5]
    # fu.dist = as.matrix(vegdist((fu.m),method="jaccard"))
    
    # clustered based on who users like
    # fu.m = fu.m[rowSums(fu.m>0)>5,colSums(fu.m>0)>5]
    # fu.m = fu.m[rowSums(fu.m>0)>5,colSums(fu.m>0)>5]
    # fu.m = fu.m[rowSums(fu.m>0)>5,colSums(fu.m>0)>5]
    # fu.m = fu.m[rowSums(fu.m>0)>5,colSums(fu.m>0)>5]
    fu.m = fu.m[intersect(colnames(fu.m),rownames(fu.m)),intersect(colnames(fu.m),rownames(fu.m))]
    fu.m = fu.m + t(fu.m)

    # clustering    
    fu.dist = as.matrix(vegdist(t(fu.m),method="jaccard"))
    fu.pca = FastPCA(fu.dist, top.k = 50)
    fu.tsne = Rtsne(fu.pca$x, pca = FALSE, is_distance = F, verbose = T, check_duplicates = F, perplexity = 30, max_iter = 5000)
    fu.hclust = hclust(dist(fu.tsne$Y),method="single")

    # k is the number of clusters to use
    fu.cut = cutree(fu.hclust, k=16)
    
    if(length(fuser.favs)>2) {
      pdf(file="twitter-ecology.pdf",width=32,height=32)
      # heatmap
      heatmap(fu.dist,hclustfun = function(x) hclust(x,method = "ward.D"),cexRow=0.1,cexCol=0.1)

      # circles sized to number of likes found per user
      plot(fu.tsne$Y,pch=21,bg=rgb(0,0,1,0.3),col=NULL,cex=0.002*((colSums(fu.m))))
      text(fu.tsne$Y, labels = colnames(fu.m),cex = 0.5)
      
      # circles sized to like count
      plot(fu.tsne$Y,pch=21,bg=alpha(gg_color_hue(max(fu.cut))[fu.cut],0.3),col=NULL,cex=0.3*sqrt(rowSums(fu.m)[colnames(fu.m)]), bty="n", axes=F, xaxt='n', yaxt='n')
      text(fu.tsne$Y, labels = colnames(fu.m),cex = 0.4)
      
      # circles sized to follower count
      plot(fu.tsne$Y,pch=21,bg=rgb(0,0,1,0.3),col=NULL,cex=0.7*sqrt(sqrt(fuser.data$followers_count[match(colnames(fu.m),fuser.data$screen_name)])))
      text(fu.tsne$Y, labels = colnames(fu.m),cex = 0.5)
      dev.off()
    }
    
    # add new users who have at least 200 likes from the group
    addusers = fu.df[order(rowSums(fu.df),decreasing=T),]
    fusers = union(fusers,rownames(addusers)[rowSums(addusers)>200])

  } else { print("sleeping"); Sys.sleep(60) }
}